# **Prerequisites**

In [ ]:
!pip install pymkv
!sudo sh -c 'echo "deb https://mkvtoolnix.download/ubuntu/ $(lsb_release -sc) main" >> /etc/apt/sources.list.d/bunkus.org.list'
!wget -q -O - https://mkvtoolnix.download/gpg-pub-moritzbunkus.txt | sudo apt-key add -!sudo apt-get -qq update
!sudo apt-get -qq install mkvtoolnix mkvtoolnix-gui
!mkvmerge -V

import os
import re
import subprocess
from pathlib import Path
from IPython.display import clear_output
import pymkv

# **Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#**Navigate to the folder containing the episodes**

*Follow the Menu till you reach the required folder,
If there's no input option, then just stop and run the cell again*

**Choose drive first, followed by either "shared drives" or "my drive" and navigate to the desired folder**



In [ ]:
current_directory = (Path.cwd()).resolve()

while True:
    items = sorted(list(current_directory.iterdir()))
    print("Current Directory: ", current_directory)
    for index, item in enumerate(items, 1):
        print('{0:2d}:'.format(index), item.name, end="\n")
  
    choice = int(input("Enter the choice (0 to stop, -1 for previous directory): "))

    if choice == 0:
        break
    
    if choice == -1:
        current_directory = current_directory.parent
    else:
        current_directory = current_directory / items[choice - 1].name

    os.chdir(current_directory)
    clear_output()

clear_output()
print("Current Directory: ", current_directory)

#**Batch Purge**

Works on the mkvfiles present in the current directory


In [ ]:
# Purges tracks based on user input
current_directory = (Path.cwd()).resolve()
output_directory = (current_directory / 'output').resolve()

def create_folder():
    '''Creates the output folder if it doesn't already exist'''

    if not output_directory.exists():
        output_directory.mkdir()
    

def demux(mkv, mkvfile, count):

    if count:
        for _ in range(count):
            mkv.remove_track(0)
        mkv.mux(output_directory / mkvfile, silent=True)
    else:
        print('No Tracks To Remove')


def display_track(track, i):
    '''Displays the tracks for a given mkvfile'''

    print('{0:02d}: TRACK NAME: {1}, TRACK TYPE: {2}, TRACK LANGUAGE: {3}'.format(i, track.track_name, track.track_type, track.language))


def remove(mkv, tracks, choice):
    '''Chooses the tracks to remove from the mkvfile'''

    print('\nTracks Present:')

    for i, track in enumerate(tracks, 1):
        display_track(track, i)

    count = 0
    TID = 0

    print('\nRemoving Tracks:')
    for track in tracks:
        try:
            if (choice == 1) and ((track.track_type == 'audio' and track.language not in ['jpn', 'und']) or \
                (track.track_type == 'subtitles' and track.language not in ['eng', 'jpn', 'und']) or \
                ('sign' in track.track_name.lower()) or ('eng' in track.track_name.lower() and track.track_type != 'subtitles')):

                TID = track.track_id
                

            elif (choice == 2) and ('commentary' in track.track_name):
                TID = track.track_id

            else:
                continue

        except Exception as e:
            print('{0:02d}: ERROR:'.format(TID + 1), e, ', SKIPPING TRACK')
            continue

        mkv.move_track_front(TID)
        count += 1
        display_track(track, TID + 1)

    return count        


def main():
    create_folder()
    mkvfiles = sorted([item for item in current_directory.iterdir() if item.suffix == '.mkv' and not item.is_dir()])

    choice = int(input('1) Remove Only Dub\n2) Remove Only Commentary\nChoice: '))

    if choice not in  [1, 2]:
        exit(0)

    for mkvfile in mkvfiles:
        print('-----------------------------------------')
        print('\nWorking File: ', mkvfile.name, end='\n\n')

        try:
            mkv = pymkv.MKVFile(mkvfile)
        except KeyError as e:
            print('ERROR:', e, 'NOT RECOGNIZED, SKIPPING CURRENT FILE')
            continue

        mkv.no_global_tags()
        tracks = mkv.get_track()

        count = remove(mkv, tracks, choice)
        demux(mkv, mkvfile.name, count)

    print('\nDone')


if __name__ == '__main__':
    main()

#**Batch Extract**

Works on the mkvfiles present in the current directory

In [ ]:
# Extracts all the tracks, chapters, tags and fonts
current_directory = (Path.cwd()).resolve()
output_directory = (current_directory / 'output').resolve()

def create_folder():
	'''Creates the output folder if it doesn't already exist'''

	if not output_directory.exists():
		output_directory.mkdir()
		

def extract_fonts(mkvfile):
	'''Extracts the fonts from the mkvfile'''

	fonts = []
	fonts_command = ''

	mkvinfo = subprocess.run('mkvmerge --identify \"{}\"'.format(mkvfile), capture_output=True, text=True, shell=True)
	pattern = re.compile(r'name \'.+(.ttf|.otf|.TTF|.OTF)\'')
	matches = pattern.finditer(mkvinfo.stdout)

	for match in matches:
		fonts.append(mkvinfo.stdout[match.start() + 6 : match.end() - 1])

	attachment_folder = (output_directory / mkvfile.name).resolve()
	font_folder = (attachment_folder / 'attachments').resolve()
	
	if not attachment_folder.exists():		
		attachment_folder.mkdir()
		font_folder.mkdir()

	for i, font in enumerate(fonts, 1):
		fonts_command += '{}:\"{}\" '.format(i, font_folder / font)
	
	if fonts_command != '':
		subprocess.run('mkvextract \"{}\" attachments {}'.format(mkvfile, fonts_command), shell=True)
	else:
		print('No fonts\n')


def extract_subs(mkvfile):
	'''Extracts the subs from the mkvfile'''

	mkv = pymkv.MKVFile(mkvfile)
	tracks = mkv.get_track()

	subtitle_command = ''

	for track in tracks:
		if track.track_type == 'subtitles':
			subtitle_command += '{}:\"{}/track{}_{}.ass\" '.format(track.track_id, output_directory / mkvfile.name, track.track_id + 1,  track.language)

	if subtitle_command != '':
		subprocess.run('mkvextract \"{}\" tracks {}'.format(mkvfile, subtitle_command), shell=True)
	else:
		print('No subtitles\n')


def extract_chapters(mkvfile):
	'''Extracts the chapter from the mkvfile'''

	subprocess.run('mkvextract \"{}\" chapters \"{}\"'.format(mkvfile, output_directory / mkvfile.name / 'chapters.xml'), shell=True)


def main():
	create_folder()
	mkvfiles = sorted([item for item in current_directory.iterdir() if item.suffix == '.mkv'])

	if mkvfiles:
		for mkvfile in mkvfiles:
			print('Working File: ', mkvfile.name, end='\n\n')

			print('\nExtracting Fonts:\n')
			extract_fonts(mkvfile)

			print('\nExtracting Chapters:\n')
			extract_chapters(mkvfile)

			print('\nExtracting Subs:\n')
			extract_subs(mkvfile)

			print("\n")


if __name__ == '__main__':
	main()

#**Batch Mux**

This script is used to batch append Chapters, fonts and subtitles into the mkvfiles.\
Follows the animetosho attachments download folder structure.\
(Created for anime use)

```
-------Folder Structure-------

root
|
|--append
|  |--attachments_folder_1
|  |  |--attachments
|  |  |  |--font1.ttf
|  |  |  `--font2.ttf
|  |  |
|  |  |--subtitle1.ass
|  |  |--subtitle2.ass
|  |  |
|  |  `--chapters.xml
|  |
|  |--attachments_folder_2
|  |  |--attachments
|  |  |  |--font1.ttf
|  |  |  `--font2.ttf
|  |  |
|  |  |--subtitle1.ass
|  |  |--subtitle2.ass
|  |  |
|  `  `--chapters.xml
|
|--mkvfiles
|  |--mkv1.mkv
|  `--mkv2.mkv
|
|--output (will be created)
|  |--mkvout1.mkv
|  `--mkvout2.mkv
|
`--Batch_Mux.py
```

The above folder structure is to be followed.

In [ ]:
# Muxes subs, attachments and chapters into a mkvfile
parent_directory = (Path.cwd()).resolve()
mkvfiles_directory = (parent_directory / 'mkvfiles').resolve()
attachments_directory = (parent_directory / 'append').resolve()
output_directory = (parent_directory / 'output').resolve()

def create_folder():
	'''Creates the output folder if it doesn't already exist'''

	if not output_directory.exists():
		output_directory.mkdir()

	if not attachments_directory.exists():
		attachments_directory.mkdir()
  

def prerequisite():
	'''Prerequisites before starting the muxing'''

	print('\nChoose the properties to ignore which are already present in the file (Remove properties in the final output)')
	print('--1) Chapters\n--2) Attachments (Fonts)\n--3) Global Tags\n--4) Track Tags')
	existing = input('Enter the numbers (Enter 0 to skip this menu): ')

	print('\nChoose the properties to ignore which are to be appended to the file (Won\'t append to the final output)')
	print('--1) Chapters\n--2) Attachments (Fonts)\n--3) Subtitles')
	insert = input('Enter the numbers (Enter 0 to skip this menu, adds all the files present in the attachments folder): ')
 
	zipfile = input('\nEnter download link for attachments (Enter 0 to skip this menu, assumes "append" folder already exists and is populated): ')
	if zipfile != '0':
		print("Downloading...")
		subprocess.run('wget "{}" -P "{}"'.format(zipfile, parent_directory), shell=True)

		for file in parent_directory.iterdir():
			if file.suffix in ['.7z', '.zip']:
				extract = file
				print("Extracting...")
				subprocess.run('7z x "{}" -o"{}"'.format(extract.name, attachments_directory), shell=True)
				break
 
	return [existing, insert]
 

def get_list(source_directory):
	'''Returns a list of items present in a directory specified by the parameter'''

	temp_list = [item for item in source_directory.iterdir()]
	return sorted(temp_list)


def append(attachments_folder, mkv, insert):
	'''Appends the fonts, subtitle tracks and chapters if present'''

	font_folders = []
	subs = []
	chapter = None

	for item in attachments_folder.iterdir():
		if item.is_dir() and '2' not in insert:
			font_folders.append(item)
		elif item.suffix == '.ass' and '3' not in insert:
			subs.append(item)
		elif ('chapters' in item.name.lower() and item.suffix == '.xml') and '1' not in insert:
			chapter = item
		else:
			continue

	if font_folders:
		print('\nAdding fonts:')
		for font_folder in font_folders:
			for font in font_folder.iterdir():
				print('--{}'.format(font.name))
				mkv.add_attachment(str(font))
	else:
		print('\nNo fonts to append')

	if subs:
		print('\nAdding subtitles:')
		for subtitle in subs:
			print('--{}'.format(subtitle.name))
			mkv.add_track(str(subtitle))
	else:
		print('\nNo subtitles to append')

	if chapter:
		print('\nAdding chapter:\n--{}'.format(chapter.name))
		mkv.chapters(str(chapter))
	else:
		print('\nNo chapters to append')


def muxing(mkv, mkvfile):
	'''Muxes the mkv file with the changes made'''

	mkv.mux(output_directory / mkvfile.name, silent=True)
	print('\nDone.')


def ignore_existing(mkv, choice):
	'''Ignore certain MKV file properties which are present in the file'''

	if '0' in choice:
		return

	print('\nIgnoring the following properties present in the file')
	if '1' in choice:
		print('--chapters')
		mkv.no_chapters()
	if '2' in choice:
		print('--attachments')
		mkv.no_attachments()
	if '3' in choice:
		print('--global tags')
		mkv.no_global_tags()
	if '4' in choice:
		print('--track tags')		
		mkv.no_track_tags()


def main():
	create_folder()
	existing, insert = prerequisite()
	mkvfiles = get_list(mkvfiles_directory)
	attachments = get_list(attachments_directory)

	for i, mkvfile in enumerate(mkvfiles):
		print('---------------------------------------------------------------------------')
		print('\nWorking File: ', mkvfile.name)
		mkv = pymkv.MKVFile(mkvfile)
		ignore_existing(mkv, existing)
		append(attachments[i], mkv, insert)
		muxing(mkv, mkvfile)


if __name__ == '__main__':
	main()
